In [14]:
import requests
import json
import random
from pprint import pprint
import pandas as pd
from pathlib import Path

# Function to get random chest_pain_values and question_values for a given key
def get_random_values(user_key, num_values, result_dict):
    if user_key in result_dict:
        df_list = result_dict[user_key]
        
        # Check if the requested number of values is greater than the available entries
        num_values = min(num_values, len(df_list))
        
        # Randomly select 'num_values' entries
        random_entries = random.sample(df_list, num_values)

        return random_entries
    else:
        return None


# Ask the user for a key
user_key = input("Enter a key from result_dict: ")

# Ask the user for the number of random values to retrieve
num_values = int(input("Enter the number of random values to retrieve: "))

result_dict = json.load(open("dict.json", "r"))

# Get random values for the specified key
random_values = get_random_values(user_key, num_values, result_dict)

# Display the result
if random_values is not None:
    print(f"Random values for key '{user_key}':")
    for i, entry in enumerate(random_values, start=1):
        print(f"{i}. Chest Pain: {entry['Chest Pain']}, Question: {entry['question']}")

    # Prepare the payload for the API request
    api_payload = {
        "all_symtoms": [{"id": entry['question'], "symtoms": entry['Chest Pain']} for entry in random_values],
        "current": [{"id": entry['question'], "symtoms": entry['Chest Pain']} for entry in random_values]
    }

    # Convert the payload to JSON
    api_payload_json = json.dumps(api_payload)

    # API request
    url = "http://192.168.0.122:8006/ekg/both_symtoms_prediction/?action=v1"
    headers = {'Content-Type': 'application/json'}
    response = requests.post(url, headers=headers, data=api_payload_json)

    # Print API response
    print("\n\n")
    print(json.dumps(response.json()))
    response_json = response.json()
    print("\n\n")
    pprint(response_json)
else:
    print(f"The key '{user_key}' is not found in result_dict.")


ValueError: invalid literal for int() with base 10: ''

In [ ]:
all_data = []

live_data = response_json["live"]["data"]
beta_data = response_json["beta"]["data"]

# pprint(live_data)
# print()
# pprint(beta_data)

live_top_diseases = [entry['diseases_name'] for entry in live_data[:2]]
beta_top_diseases = [entry['diseases_name'] for entry in beta_data[:2]]


print("live_top_diseases: ",live_top_diseases)
print("beta_top_diseases: ", beta_top_diseases)

status_live = "pass" if user_key in live_top_diseases else "fail"
status_beta = "pass" if user_key in beta_top_diseases else "fail"

print("Input: ",user_key)
print("status_live: ", status_live)
print('status_beta: ', status_beta)


headers = ["live/beta", "desease", "status", "model_output", "symptoms"]
row_live = ["live", user_key, status_live, ', '.join(desease for desease in live_top_diseases),  ', '.join(entry['Chest Pain'] for entry in random_values)]
row_beta = ["beta", user_key, status_beta, ', '.join(desease for desease in beta_top_diseases),  ', '.join(entry['Chest Pain'] for entry in random_values)]

print(len(headers))
print(len(row_live))
print(len(row_beta))

csv_file_path = "test.csv"
rows = [row_live, row_beta]

df= pd.DataFrame(rows, columns=headers)
if Path(csv_file_path).exists():
    existing_df = pd.read_csv(csv_file_path)
    updated_df = pd.concat([existing_df, df],  ignore_index=True)
    updated_df.to_csv(csv_file_path, index = False)
else:
    df.to_csv(csv_file_path, index= False)

NameError: name 'response_json' is not defined

In [ ]:
import requests
import json
import random
from pprint import pprint
import pandas as pd
from pathlib import Path

diseases = ['Aortic Dissection', 'AS/AI', 'CHF', 'Eso rupture', 'GI', 'infarct', 'Ischemia', 'MS', 'PE', 'Pericarditis', 'Pulmonary']
num_tests = 50


# Function to get random chest_pain_values and question_values for a given key
def get_random_values(user_key, num_values, result_dict):
    if user_key in result_dict:
        df_list = result_dict[user_key]
        # Check if the requested number of values is greater than the available entries
        num_values = min(num_values, len(df_list))
        # Randomly select 'num_values' entries
        random_entries = random.sample(df_list, num_values)
        return random_entries
    else:
        return None


for test_num in range(1, num_tests + 1):
    print("**************** START ***********************")

    user_key = random.choice(diseases)
    print("Input: ",user_key)

    num_values = random.randint(5, 8)
    print("num_values (5 to 8):  ",num_values)

    random_values = get_random_values(user_key, num_values, result_dict)
    
    api_payload = {
            "all_symtoms": [{"id": entry['question'], "symtoms": entry['Chest Pain']} for entry in random_values],
            "current": [{"id": entry['question'], "symtoms": entry['Chest Pain']} for entry in random_values]
        }

    # Convert the payload to JSON
    api_payload_json = json.dumps(api_payload)

    url = "http://192.168.0.122:8006/ekg/both_symtoms_prediction/?action=v1"
    headers = {'Content-Type': 'application/json'}
    response = requests.post(url, headers=headers, data=api_payload_json)
    
    response_json = response.json()

    all_data = []

    live_data = response_json["live"]["data"]
    beta_data = response_json["beta"]["data"]

    live_top_diseases = [entry['diseases_name'] for entry in live_data[:2]]
    beta_top_diseases = [entry['diseases_name'] for entry in beta_data[:2]]

    print("live_top_diseases: ",live_top_diseases)
    print("beta_top_diseases: ", beta_top_diseases)

    status_live = "pass" if user_key in live_top_diseases else "fail"
    status_beta = "pass" if user_key in beta_top_diseases else "fail"

    print("Input: ",user_key)
    print("status_live: ", status_live)
    print('status_beta: ', status_beta)


    headers = ["live/beta", "desease", "status", "model_output", "symptoms"]
    row_live = ["live", user_key, status_live, ', '.join(desease for desease in live_top_diseases),  ', '.join(entry['Chest Pain'] for entry in random_values)]
    row_beta = ["beta", user_key, status_beta, ', '.join(desease for desease in beta_top_diseases),  ', '.join(entry['Chest Pain'] for entry in random_values)]

    csv_file_path = "test.csv"
    rows = [row_live, row_beta]

    df= pd.DataFrame(rows, columns=headers)
    if Path(csv_file_path).exists():
        existing_df = pd.read_csv(csv_file_path)
        updated_df = pd.concat([existing_df, df],  ignore_index=True)
        updated_df.to_csv(csv_file_path, index = False)
    else:
        df.to_csv(csv_file_path, index= False)
    

    print("**************** COMPLETE ***********************")


**************** COMPLETE ***********************
Input:  Ischemia
num_values (5 to 8):   6
live_top_diseases:  ['Infarct', 'Ischemia']
beta_top_diseases:  ['Ischemia', 'Infarct']
Input:  Ischemia
status_live:  pass
status_beta:  pass
**************** COMPLETE ***********************
**************** COMPLETE ***********************
Input:  MS
num_values (5 to 8):   6
live_top_diseases:  ['MS', 'Pericarditis']
beta_top_diseases:  ['MS', 'GI']
Input:  MS
status_live:  pass
status_beta:  pass
**************** COMPLETE ***********************
**************** COMPLETE ***********************
Input:  Eso rupture
num_values (5 to 8):   8
live_top_diseases:  ['Pulmonary', 'CHF']
beta_top_diseases:  ['Pulmonary', 'CHF']
Input:  Eso rupture
status_live:  fail
status_beta:  fail
**************** COMPLETE ***********************
**************** COMPLETE ***********************
Input:  CHF
num_values (5 to 8):   5
live_top_diseases:  ['CHF', 'Pulmonary']
beta_top_diseases:  ['CHF', 'Pulmonary']


KeyError: 'live'

In [ ]:

import requests
import json
import random
import time
from pprint import pprint
import pandas as pd
from pathlib import Path



result_dict = json.load(open("dict.json", "r"))
diseases = ['Aortic Dissection', 'AS/AI', 'CHF', 'Eso rupture', 'GI', 'infarct', 'Ischemia', 'MS', 'PE', 'Pericarditis', 'Pulmonary']


# Function to get random chest_pain_values and question_values for a given key
def get_random_values(user_key, num_values, result_dict):
    if user_key in result_dict:
        df_list = result_dict[user_key]
        # Check if the requested number of values is greater than the available entries
        num_values = min(num_values, len(df_list))
        # Randomly select 'num_values' entries
        random_entries = random.sample(df_list, num_values)

        age_count = 0
        filtered_entries = []

        for entry in random_entries:
            if 'Age' in entry['Chest Pain'] and age_count == 0:
                filtered_entries.append(entry)
                age_count += 1
            elif 'Age' not in entry['Chest Pain']:
                filtered_entries.append(entry)

        # inline loop
        # filtered_entries = [entry for entry in random_entries if ('Age' in entry['Chest Pain'] and (age_count := age_count + 1) == 1) or 'Age' not in entry['Chest Pain']]        
        return filtered_entries
    else:
        return None





/tmp/ipykernel_52581/2736719613.py:6: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [ ]:
base_url= "http://127.0.0.1:8000"

def get_symptoms(diseases_name, version):
    if version == "v1":
        slice_value = 4
    if version == "v2":
        slice_value = 1
    if version == "v3":
        slice_value = 2
    if version == "v4":
        slice_value = 4
    
    url = f"{base_url}/ekg/scripting/?disease_name={diseases_name}&version={version}"

    payload = {}
    headers = {}
    response = requests.request("GET", url, headers=headers, data=payload)
    resp_data= json.loads(response.text)
    data= resp_data['data']
    print(len(data))
    print(data)
    slice_data= data[:slice_value]
    print(len(slice_data))
    print(slice_data)


    api_payload = {
            "all_symtoms": [{"id": entry['question_id'], "symtoms": entry['name']} for entry in slice_data],
            "current": [{"id": entry['question_id'], "symtoms": entry['name']} for entry in slice_data]
        }

    print(json.dumps(api_payload, indent=2))

get_symptoms("Infarct", "v3")

19
[{'name': 'DM (Diabetes mellitus)', 'probability': 0.95, 'question_id': '881ba1b1-1c32-4b1c-b02f-203f0487e20a'}, {'name': "GreaterCRF's (Chronic renal failure)", 'probability': 0.9, 'question_id': '881ba1b1-1c32-4b1c-b02f-203f0487e20a'}, {'name': 'TOB (Tobacco)', 'probability': 0.9, 'question_id': 'c60b0a04-c681-4e18-874d-5e235470d6c4'}, {'name': 'Age<20', 'probability': 0.9, 'question_id': 'ad9b04e6-e4f5-4e3a-bd5d-eb2ab1acb75e'}, {'name': 'Age>65', 'probability': 0.9, 'question_id': 'ad9b04e6-e4f5-4e3a-bd5d-eb2ab1acb75e'}, {'name': 'HTN (Hypertension)', 'probability': 0.8, 'question_id': '881ba1b1-1c32-4b1c-b02f-203f0487e20a'}, {'name': 'CP1Day (cytopathologists)', 'probability': 0.7, 'question_id': '9018fc95-6eb2-4db5-8a4c-2ec88fa74323'}, {'name': 'HLD (Hyperlipidemia)', 'probability': 0.65, 'question_id': '881ba1b1-1c32-4b1c-b02f-203f0487e20a'}, {'name': 'FH (Familial history)', 'probability': 0.65, 'question_id': '9c1eac58-d947-42db-811a-d41a82171cfc'}, {'name': 'Age<60', 'proba

In [ ]:

# Number of test cases
num_tests = 1

for test_num in range(1, num_tests + 1):
    # time.sleep(1)

    print(f"\n\n**************** START {test_num} ***********************")

    user_key = random.choice(diseases)
    print("Input: ",user_key)

    num_values = random.randint(5, 8)
    print("num_values (5 to 8):  ",num_values)

    random_values = get_random_values(user_key, num_values, result_dict)
    print("\n\nRamdom Val", json.dumps(random_values, indent=2))

    api_payload = {
            "all_symtoms": [{"id": entry['question'], "symtoms": entry['Chest Pain']} for entry in random_values],
            "current": [{"id": entry['question'], "symtoms": entry['Chest Pain']} for entry in random_values]
        }
    print("\n\nAPI PAYLOAD\n", json.dumps(api_payload, indent=2))



**************** START 1 ***********************
Input:  infarct
num_values (5 to 8):   6


Ramdom Val [
  {
    "Chest Pain": "Dull",
    "question": "7bd78b32-03c0-4c28-9208-f4e9c2967558"
  },
  {
    "Chest Pain": "Squeezing",
    "question": "7bd78b32-03c0-4c28-9208-f4e9c2967558"
  },
  {
    "Chest Pain": "Central",
    "question": "9cf10e77-00db-4ea9-bdca-c2e0d04c101b"
  },
  {
    "Chest Pain": "Sbp>140 (systolic blood pressure is greater than 140)",
    "question": "ae18f576-c22e-4c27-a1ef-32e2401e1285"
  },
  {
    "Chest Pain": "Exertional",
    "question": "9cf10e77-00db-4ea9-bdca-c2e0d04c101b"
  },
  {
    "Chest Pain": "Nausea",
    "question": "0cf99e50-3b21-4219-b921-80d062d5bf5c"
  }
]


API PAYLOAD
 {
  "all_symtoms": [
    {
      "id": "7bd78b32-03c0-4c28-9208-f4e9c2967558",
      "symtoms": "Dull"
    },
    {
      "id": "7bd78b32-03c0-4c28-9208-f4e9c2967558",
      "symtoms": "Squeezing"
    },
    {
      "id": "9cf10e77-00db-4ea9-bdca-c2e0d04c101b",
      "sym

In [ ]:
base_url= "http://127.0.0.1:8000"

def get_symptoms(diseases_name, version):
    if version == "v1":
        slice_value = 4
    if version == "v2":
        slice_value = 1
    if version == "v3":
        slice_value = 2
    if version == "v4":
        slice_value = 4
    
    url = f"{base_url}/ekg/scripting/?disease_name={diseases_name}&version={version}"

    payload = {}
    headers = {}
    response = requests.request("GET", url, headers=headers, data=payload)
    resp_data= json.loads(response.text)
    data= resp_data['data']
    print(len(data))
    print(data)
    slice_data= data[:slice_value]
    print(len(slice_data))
    print(slice_data)


    api_payload = {
            "all_symtoms": [{"id": entry['question_id'], "symtoms": entry['name']} for entry in slice_data],
            "current": [{"id": entry['question_id'], "symtoms": entry['name']} for entry in slice_data]
        }

    print(json.dumps(api_payload))
    # Convert the payload to JSON
    api_payload_json = json.dumps(api_payload)

    url = f"{base_url}/ekg/both_symtoms_prediction/?action=v1"
    headers = {'Content-Type': 'application/json'}
    # time.sleep(1)
    response = requests.post(url, headers=headers, data=api_payload_json, timeout=300)
    # time.sleep(2)
    
    response_json = response.json()
    return response_json


response = get_symptoms("Infarct", "v4")

print(response)

23
[{'name': 'Diaphoresis', 'probability': 0.7, 'question_id': '1b2d7a50-a47c-40c3-afd5-cce46c52ac43'}, {'name': 'HR>100 (heart rate greater than 100)', 'probability': 0.7, 'question_id': 'ae18f576-c22e-4c27-a1ef-32e2401e1285'}, {'name': 'Sbp>140 (systolic blood pressure is greater than 140)', 'probability': 0.7, 'question_id': 'ae18f576-c22e-4c27-a1ef-32e2401e1285'}, {'name': 'Sbp< 100 (systolic blood pressure is less than 100)', 'probability': 0.5, 'question_id': 'ae18f576-c22e-4c27-a1ef-32e2401e1285'}, {'name': 'CTA (Computed Tomography Angiography)', 'probability': 0.5, 'question_id': 'b067db34-139f-40b9-841b-7347a464ec0e'}, {'name': 'JVD> 4cm (jugular venous distension greater than 4 centimeters)', 'probability': 0.3, 'question_id': '1b2d7a50-a47c-40c3-afd5-cce46c52ac43'}, {'name': 'RR>20 (respiratory rate is greater than 20)', 'probability': 0.3, 'question_id': 'ae18f576-c22e-4c27-a1ef-32e2401e1285'}, {'name': 'Rales', 'probability': 0.3, 'question_id': 'b067db34-139f-40b9-841b-7

In [77]:
import requests
import json
import random

BASE_URL = "http://127.0.0.1:8000"

def get_symptoms(diseases_name: str, version: str):
    version_data = {
        'v1': 4,
        'v2': 1,
        'v3': 2,
        'v4': 4 
    }
    
    if version in version_data:
        slice_value = version_data[version]
        url = f"{BASE_URL}/ekg/scripting/?disease_name={diseases_name}&version={version}"

        response = requests.request("GET", url)
        resp_data = json.loads(response.text)
        data = resp_data['data']
        print(len(data))

        # Separating age-related symptoms and non-age-related symptoms
        age_symptoms = [symptom for symptom in data if 'Age' in symptom['name']]
        other_symptoms = [symptom for symptom in data if 'Age' not in symptom['name']]

        # Randomly selecting one age-related symptom if available
        if age_symptoms:
            selected_symptoms = random.sample(age_symptoms, 1) + random.sample(other_symptoms, min(slice_value - 1, len(other_symptoms)))
        else:
            selected_symptoms = random.sample(data, min(slice_value, len(data)))

        print(len(selected_symptoms))

        if len(selected_symptoms) == 0:
            print(url)
            print(resp_data)

        return selected_symptoms
    else:
        print(f"Problem in {version}  only {version_data} is accepted ....")

# Example usage:
get_symptoms("CHF", "v3")


19
2


[{'name': 'Age<60',
  'probability': 0.5,
  'question_id': 'ad9b04e6-e4f5-4e3a-bd5d-eb2ab1acb75e'},
 {'name': "GreaterCRF's (Chronic renal failure)",
  'probability': 0.2,
  'question_id': '881ba1b1-1c32-4b1c-b02f-203f0487e20a'}]

In [144]:

import requests
import json
import random
import time
from pprint import pprint
import pandas as pd
from pathlib import Path

DISEASES = ['Aortic Dissection', 'AS/AI', 'CHF', 'Eso rupture', 'GI', 'Infarct', 'Ischemia', 'MS', 'PE', 'Pericarditis', 'Pulmonary']
BASE_URL= "http://127.0.0.1:8000"




def get_symptoms(diseases_name: str, version: str):
    version_data = {
        'v1': 4,
        'v2': 1,
        'v3': 2,
        'v4': 4 
    }
    
    if version in version_data:
        slice_value = version_data[version]
        url = f"{BASE_URL}/ekg/scripting/?disease_name={diseases_name}&version={version}"

        response = requests.request("GET", url)
        resp_data = json.loads(response.text)
        data = resp_data['data']
        print(len(data))

        # Separating age-related symptoms and non-age-related symptoms
        age_symptoms = [symptom for symptom in data if 'Age' in symptom['name']]
        other_symptoms = [symptom for symptom in data if 'Age' not in symptom['name']]

        # Randomly selecting one age-related symptom if available
        if age_symptoms:
            selected_symptoms = random.sample(age_symptoms, 1) + random.sample(other_symptoms, slice_value - 1)
        else:
            selected_symptoms = random.sample(data, slice_value)

        print(len(selected_symptoms))

        if len(selected_symptoms) == 0:
            print(url)
            print(resp_data)

        return selected_symptoms
    else:
        print(f"Problem in {version}  only {version_data} is accepted ....")





# Number of test cases
num_tests = 2

for test_num in range(1, num_tests + 1):
    # time.sleep(1)

    print(f"\n\n**************** START {test_num} ***********************")

    random_disease = random.choice(DISEASES)
    print("Input: ",random_disease)

    # all symptoms from all version v1 to v4
    all_symptoms = []

    versions= ["v1", "v2", "v3", "v4"]
    
    for version in versions:
        print(f"\n\n\t********* START {version} *********")
        # time.sleep(1)
        symptoms = get_symptoms(diseases_name= random_disease, version= version)
        all_symptoms.extend(symptoms)

    print("all_symptoms LENGTH ",len(all_symptoms))



**************** START 1 ***********************
Input:  Ischemia


	********* START v1 *********
59
4


	********* START v2 *********
16
1


	********* START v3 *********
19
2


	********* START v4 *********
23
4
all_symptoms LENGTH  11


**************** START 2 ***********************
Input:  CHF


	********* START v1 *********
59
4


	********* START v2 *********
16
1


	********* START v3 *********
19
2


	********* START v4 *********
23
4
all_symptoms LENGTH  11


In [145]:
"""
SLICE Symptoms  
1 Age 
and other symptoms

"""
import requests
import json
import random
import time
from pprint import pprint
import pandas as pd
from pathlib import Path

DISEASES = ['Aortic Dissection', 'AS/AI', 'CHF', 'Eso rupture', 'GI', 'Infarct', 'Ischemia', 'MS', 'PE', 'Pericarditis', 'Pulmonary']
BASE_URL= "http://127.0.0.1:8000"


import requests
import json
import random

BASE_URL = "http://127.0.0.1:8000"

def get_symptoms(diseases_name: str, version: str):
    version_data = {
        'v1': 4,
        'v2': 1,
        'v3': 2,
        'v4': 4 
    }
    
    slice_value = version_data.get(version, None)
    
    if slice_value is not None:
        url = f"{BASE_URL}/ekg/scripting/?disease_name={diseases_name}&version={version}"

        response = requests.request("GET", url)
        resp_data = json.loads(response.text)
        data = resp_data['data']
        print(len(data))
        
        # Separating age-related symptoms and non-age-related symptoms
        age_symptoms = [symptom for symptom in data if 'Age' in symptom['name']]
        other_symptoms = [symptom for symptom in data if 'Age' not in symptom['name']]

        selected_age_symptom = age_symptoms[0] if age_symptoms else None
        
        if selected_age_symptom:
            selected_symptoms = [selected_age_symptom] + other_symptoms[: slice_value - 1]
        else:
            selected_symptoms = data[:slice_value]

        print(len(selected_symptoms))

        if len(selected_symptoms) == 0:
            print(url)
            print(resp_data)

        return selected_symptoms
    else:
        print(f"Problem in {version}  only {version_data} is accepted ....")



get_symptoms("CHF", "v3")

19
2


[{'name': 'Age<20',
  'probability': 0.9,
  'question_id': 'ad9b04e6-e4f5-4e3a-bd5d-eb2ab1acb75e'},
 {'name': 'CP1Day (cytopathologists)',
  'probability': 0.8,
  'question_id': '9018fc95-6eb2-4db5-8a4c-2ec88fa74323'}]

In [140]:
"""
Random Select 
1 Age 
and other symptoms

"""


import requests
import json
import random
import time
from pprint import pprint
import pandas as pd
from pathlib import Path

DISEASES = ['Aortic Dissection', 'AS/AI', 'CHF', 'Eso rupture', 'GI', 'Infarct', 'Ischemia', 'MS', 'PE', 'Pericarditis', 'Pulmonary']
BASE_URL= "http://127.0.0.1:8000"
THRESHOLD_PROBABILITY = 0.6



def get_symptoms(diseases_name: str, version: str):
    version_data = {
        'v1': 4,
        'v2': 1,
        'v3': 2,
        'v4': 4 
    }
    
    if version in version_data:
        slice_value = version_data[version]
        url = f"{BASE_URL}/ekg/scripting/?disease_name={diseases_name}&version={version}"

        response = requests.request("GET", url)
        resp_data = json.loads(response.text)
        data = resp_data['data']
        print(len(data))

        # Separating age-related symptoms and non-age-related symptoms
        age_symptoms = [symptom for symptom in data if 'Age' in symptom['name']]
        other_symptoms = [symptom for symptom in data if 'Age' not in symptom['name'] and symptom['probability'] >= THRESHOLD_PROBABILITY]
        # print("\n\n", json.dumps(other_symptoms, indent=2), "\n\n")
        # Randomly selecting one age-related symptom if available
        
        if age_symptoms:
            if len(other_symptoms) >= slice_value -1 :
                selected_symptoms = random.sample(age_symptoms, 1) + random.sample(other_symptoms, slice_value - 1)
            else:
                print("______________________ PROBLEM 1 ")
                other_symptoms = [symptom for symptom in data if 'Age' not in symptom['name']]
                selected_symptoms = random.sample(age_symptoms, 1) + other_symptoms[:slice_value - 1]
                
        else:
            if len(other_symptoms) >= slice_value :
                selected_symptoms = random.sample(other_symptoms, slice_value)
            else:
                print("______________________ PROBLEM 2 ")
                other_symptoms = [symptom for symptom in data if 'Age' not in symptom['name']]
                selected_symptoms = other_symptoms[:slice_value]

        print(len(selected_symptoms))

        if len(selected_symptoms) == 0:
            print(url)
            print(resp_data)

        return selected_symptoms
    else:
        print(f"Problem in {version}  only {version_data} is accepted ....")


# v1= get_symptoms("CHF", "v1")
# v2= get_symptoms("CHF", "v2")
v3= get_symptoms("CHF", "v3")
# v4= get_symptoms("CHF", "v4")

# print(json.dumps(v1, indent=2))
# print(json.dumps(v2, indent=2))
print(json.dumps(v3, indent=2))
# print(json.dumps(v4, indent=2))


19
2
[
  {
    "name": "Age<20",
    "probability": 0.9,
    "question_id": "ad9b04e6-e4f5-4e3a-bd5d-eb2ab1acb75e"
  },
  {
    "name": "CP1Day (cytopathologists)",
    "probability": 0.8,
    "question_id": "9018fc95-6eb2-4db5-8a4c-2ec88fa74323"
  }
]
